In [ ]:
%pip install wandb -Uqq

In [ ]:
%pip install nmslib -Uqq

     |████████████████████████████████| 13.5 MB 3.2 MB/s 
     |████████████████████████████████| 188 kB 48.8 MB/s 


In [ ]:
%pip install LibRecommender -Uqq

     |████████████████████████████████| 2.0 MB 6.6 MB/s 
     |████████████████████████████████| 24.1 MB 1.2 MB/s 


In [ ]:
%pip install faiss==1.5.3 -Uqq

     |████████████████████████████████| 4.7 MB 8.5 MB/s 


In [ ]:
%pip install ujson redis -Uqq

     |████████████████████████████████| 246 kB 8.4 MB/s 


In [ ]:
import wandb
wandb.login()

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

from libreco.data import random_split, split_by_ratio_chrono, DatasetPure
from libreco.algorithms import SVDpp, RNN4Rec  # pure data, algorithm SVD++
from libreco.evaluation import evaluate

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
from libserving.serialization import save_embed

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/00-MTC/models/RNN4Rec" 
MODEL_NAME = 'RNN4Rec'
config = {
  "embed_size": 16,
   'n_epochs': 5,
   "lr": 0.01,
    "batch_size": 64
}

In [ ]:
data = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
data.columns = ["user", "item", "label", "time"]
data["label"] = 1 

metrics = ["loss", "roc_auc", "precision", "recall", "map", "ndcg"]
train_data, eval_data, test_data = split_by_ratio_chrono(data, multi_ratios=[0.8, 0.1, 0.1])

train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
train_data.build_negative_samples(data_info)  # sample negative items for each record
eval_data.build_negative_samples(data_info) 
test_data.build_negative_samples(data_info)
print(data_info)   # n_users: 5894, n_items: 3253, data sparsity: 0.4172 %

random neg item sampling elapsed: 0.314s
random neg item sampling elapsed: 0.019s
random neg item sampling elapsed: 0.041s
n_users: 610, n_items: 8237, data sparsity: 1.6056 %


In [ ]:
from IPython.lib.display import exists
with wandb.init() as runwb:
    # Overwrite the random run names chosen by wandb
    runwb.config.update(config)
    sc = runwb.config
    name_str = f'{MODEL_NAME}' + \
      f'_n_epoch_{sc.n_epochs}_' + \
      f'emb_size_{sc.embed_size}_lr_{sc.lr}_' + \
      f'batch_size_{sc.batch_size}'
    runwb.config.update({"name": name_str})
    current_dir = os.path.join(MODEL_PATH, name_str)
    if not os.path.isdir(current_dir):
        os.mkdir(current_dir)        
    
    # Rest of code.
    model = RNN4Rec(task="ranking", data_info=data_info, 
                    embed_size=sc.embed_size, n_epochs=sc.n_epochs, 
                    lr=sc.lr, reg=None, batch_size=sc.batch_size)
    model.fit(runwb, train_data, verbose=2, eval_data=eval_data, metrics=metrics)
    evaluate(runwb=runwb, model=model, data=test_data,
                                metrics=metrics)
    # specify model saving directory
    save_embed(current_dir, model)        
    model.save(path=current_dir, model_name="RNN4Rec", manual=True, inference_only=True)

total params: 274,221 | embedding params: 271,917 | network params: 2,304
Training start time: 2022-11-05 15:51:51


train: 100%|██████████| 2521/2521 [00:39<00:00, 64.32it/s]


Epoch 1 elapsed: 39.259s
	 train_loss: 0.4957


eval_rec: 100%|██████████| 610/610 [00:00<00:00, 2611.96it/s]


	 eval log_loss: 0.5327
	 eval roc_auc: 0.8199
	 eval precision@10: 0.0300
	 eval recall@10: 0.0339
	 eval map@10: 0.0802
	 eval ndcg@10: 0.1154


...

Epoch 5 elapsed: 38.736s
	 train_loss: 0.2293


eval_rec: 100%|██████████| 610/610 [00:00<00:00, 2463.49it/s]


	 eval log_loss: 0.8101
	 eval roc_auc: 0.7609
	 eval precision@10: 0.0267
	 eval recall@10: 0.0275
	 eval map@10: 0.0620
	 eval ndcg@10: 0.0937


eval_rec: 100%|██████████| 608/608 [00:00<00:00, 2723.64it/s]


epoch,▁▃▅▆█
loss,▁
map,▁
map@10,█▁▄▅▁
ndcg,▁
ndcg@10,█▂▅▆▁
precision,▁
precision@10,▇▂▆█▁
recall,▁
recall@10,▇▆▆█▁
roc_auc,█▅▁▂▃▁


In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/00-MTC/models/RNN4Rec" 
MODEL_NAME = 'RNN4Rec'
sweep_config = {
    "name": "RNN4Rec-sweep",
    "metric": {"name": "loss", "goal": "minimize"},    
    "method": "grid",    
    "parameters": {        
          "embed_size": {
              "values": [16, 32]
          },
          'n_epochs': {
              "values": [5,10]
          }, 
          "lr": {
              "values": [0.1, 0.01, 0.001, 0.0001]
          },          
          "batch_size": {
              "values": [64, 128, 256]
          },
          }
}
sweep_id = wandb.sweep(sweep_config, 
                       project="recsys-app", 
                       entity="iloncka-ds")
# os.environ["WANDB_NOTEBOOK_NAME"] = "part-4-embedding-RNN4Rec-time-ranking-hyperparameters-search.ipynb"     
# os.environ["WANDB_RESUME"] = "allow"
# os.environ["WANDB_RUN_ID"] = wandb.util.generate_id()
# os.environ["WANDB_RUN_GROUP"] = "experiment-" + wandb.util.generate_id()

# sweep_config, resume=os.environ["WANDB_RESUME"],
#                       group=MODEL_NAME,
#                       notes=f"{MODEL_NAME} hyperparameters search",
#                       tags=["embed", "sequence", "hyperparameters"]
def train_with_wandb():  
      with wandb.init() as runwb:
        # Overwrite the random run names chosen by wandb
        sc = runwb.config
        name_str = f'{MODEL_NAME}_met_{sweep_config["method"]}' + \
          f'_n_epoch_{sc.n_epochs}_' + \
          f'emb_size_{sc.embed_size}_lr_{sc.lr}_' + \
          f'batch_size_{sc.batch_size}'
        runwb.config.update({"name": name_str})
        current_dir = os.path.join(MODEL_PATH, name_str)
        if not os.path.isdir(current_dir):
            os.mkdir(current_dir)  
        # Rest of code.
        model = RNN4Rec(task="ranking", data_info=data_info, 
                        embed_size=sc.embed_size, n_epochs=sc.n_epochs, 
                        lr=sc.lr, reg=None, batch_size=sc.batch_size)
        #NB!! param runwb is absent in official version of LibRecommender, for training unpublished changes to this library were used
        model.fit(runwb, train_data, verbose=2, eval_data=eval_data, metrics=metrics)
        #NB!! param runwb is absent in official version of LibRecommender, for training unpublished changes to this library were used
        evaluate(runwb=runwb, model=model, data=test_data,
                                    metrics=metrics)
        # specify model saving directory
        save_embed(current_dir, model)        
        model.save(path=current_dir, model_name="RNN4Rec", manual=True, inference_only=True)
wandb.agent(sweep_id, function=train_with_wandb, entity="iloncka-ds", project="recsys-app", count=20)

Create sweep with ID: tw52paxu
Sweep URL: https://wandb.ai/iloncka-ds/recsys-app/sweeps/tw52paxu


wandb: Agent Starting Run: apubg7az with config:
wandb: 	batch_size: 64
wandb: 	embed_size: 16
wandb: 	lr: 0.1
wandb: 	n_epochs: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: iloncka-ds. Use `wandb login --relogin` to force relogin


total params: 274,221 | embedding params: 271,917 | network params: 2,304
Training start time: 2022-11-05 16:00:13


train: 100%|██████████| 2521/2521 [00:39<00:00, 63.68it/s]


Epoch 1 elapsed: 39.647s
	 train_loss: 0.7138


...

Epoch 10 elapsed: 25.926s
	 train_loss: 0.0975


eval_rec: 100%|██████████| 610/610 [00:00<00:00, 1714.59it/s]


	 eval log_loss: 1.3047
	 eval roc_auc: 0.7207
	 eval precision@10: 0.0230
	 eval recall@10: 0.0273
	 eval map@10: 0.0529
	 eval ndcg@10: 0.0825


eval_rec: 100%|██████████| 608/608 [00:00<00:00, 2066.40it/s]


epoch,▁▂▃▃▄▅▆▆▇█
loss,▁
map,▁
map@10,██▅▇▆▅▁▂▄▁
ndcg,▁
ndcg@10,█▇▅▆▆▄▁▂▃▁
precision,▁
precision@10,██▇▆▆▃▁▂▂▁
recall,▁
recall@10,▇▆▆▇█▄▃▃▃▁
roc_auc,█▅▃▄▃▂▂▃▃▂▁


https://wandb.ai/iloncka-ds/recsys-app/sweeps/tw52paxu?workspace=user-iloncka-ds